In [1]:
import re
from math import sin, cos, acos, pi
from decimal import Decimal
import pandas as pd

In [2]:
geo_dct = {'lat':[],'lon':[],'ts':[]}
ctrl_dct = {'control_switch_on':[],'ts':[]}

In [3]:
# Процедура парсинга
def control_parse(s, dct):
    pattern_ctrl = ':[a-z]+'
    pattern_ts1 = ':[0-9]+' 
    ctrl_type = re.findall(pattern_ctrl, s)
    ts = re.findall(pattern_ts1, s)
    dct['control_switch_on'].append(ctrl_type[0][1:])
    dct['ts'].append(int(ts[0][1:]))

In [4]:
# Процедура парсинга
def geo_parse(s, dct):
    pattern_lat = '"lat":[0-9]*[.]*[0-9]*' 
    pattern_lon = '"lon":[-]*[0-9]*[.]*[0-9]*'
    pattern_ts2 = '"ts":[0-9]+'
    lat = re.findall(pattern_lat,s)
    lon = re.findall(pattern_lon,s)
    ts = re.findall(pattern_ts2,s)
    dct['lat'].append(Decimal(lat[0][6:]))
    dct['lon'].append(Decimal(lon[0][6:]))
    dct['ts'].append(Decimal(ts[0][5:]))

In [5]:
# Выгрузка данных и парсинг
with open('C:/Users/Andrei/Python files/','r') as file:
    for line in file:
        if line[2] == 'c':
            control_parse(line, ctrl_dct)
        else:
            geo_parse(line, geo_dct)

In [6]:
# Создание df для geo
geo_df = pd.DataFrame.from_dict(geo_dct)
geo_df = geo_df.sort_values(by='ts', ascending=True) # сортировка по времени

# Очистка от нулевых данных
zero_index = geo_df[geo_df['lon']==0].index.values.tolist()
geo_df.drop(index=zero_index, inplace = True)
geo_df.reset_index(drop = True, inplace = True) # cбрасываем индексы

In [8]:
# Создание df для control_switch_on
ctrl_df = pd.DataFrame.from_dict(ctrl_dct)
booleanDictionary = {'true': True, 'false': False}
ctrl_df['control_switch_on'] = ctrl_df['control_switch_on'].map(booleanDictionary)
ctrl_df = ctrl_df.sort_values(by='ts', ascending=True) # сортировка по времени
ctrl_df.reset_index(drop = True, inplace = True) # cбрасываем индексы

In [9]:
# Создается вспомогательный df, который будет содержать режимы переключения между pnt1 и pnt2
# На основе этого df будет сделан подсчёт времени нахождения в каждом из режимов
def support_df(t1, t2, lst, dfc):
    df = dfc.loc[lst,:]
    df.reset_index(drop = True, inplace = True) # Появились дубли индексов, поэтому обновляем
    new_ind = df.index.values.tolist()
    
    # Обновляем время начала и конца
    df.loc[new_ind[0],'ts'] = t1 
    df.loc[new_ind[-1],'ts'] = t2
    return df   

In [10]:
# Процедура подсчёта времени управления каждого типа
def time_slots(pnt1, pnt2, dfc, i_search):    

    # Индекс до pnt1
    ind_bellow = [] 
    if i_search == -1: # (начало процесса, когда меток времени может и не быть)
        i = 0
        while (i< dfc.shape[0]) and (dfc.loc[i,'ts']<=pnt1.ts):
            ind_bellow = [i]
            i += 1
    else:
        ind_bellow = [i_search] # Nая итерация - берём результаты предыдущего шага
    
    
    # Индексы  между pnt1, pnt2
    ind = [] 
    i = i_search+1
    while (i< dfc.shape[0]) and (dfc.loc[i,'ts']<=pnt2.ts):
        ind.append(i)
        i += 1   
    
    
    # Возможны следующие ситуации:
    # 1. данных по режиму нет: geo1_TS --> geo2_TS
    # 2. данные по режиму появляются в середине: geo1_TS --> control_switch_on1_TS --> geo2_TS 
    # 3. данные по режиму есть только до: control_switch_on1_TS --> geo1_TS --> geo2_TS
    # 4. данные по режиму есть везде: control_switch_on1_TS --> geo1_TS --> control_switch_on2_TS (может быть много переключений) --> geo2_TS 
    
    # В этой переменной считаем время управления ручного, автоматического и неизвестного типа:
    c_dct = {None:[], True:[], False:[]} 
    
    # 1.
    if ind_bellow == [] and ind == []:
        #print('Ситуация1')
        c_dct[None]=[pnt2.ts-pnt1.ts]
        i_search = -1
        return (c_dct, i_search)
        
    # 2.
    if ind_bellow == [] and ind != []:
        #print('Ситуация2')
        mid_ind = ind[0]
        c_dct[None]=[dfc.loc[mid_ind,'ts'] - pnt1.ts]
        
        # Список индексов
        last_ind = [ind[-1]]         
        ind = ind + last_ind
        
        result_df = support_df(dfc.loc[mid_ind,'ts'], pnt2.ts, ind, dfc)
        new_ind = result_df.index.values.tolist() 
        # Считаем время управления ручного и автоматического
        for i in new_ind[0:-1]:
            c_dct[result_df.loc[i,'control_switch_on']].append(result_df.loc[i+1,'ts'] - result_df.loc[i,'ts'])
        i_search = last_ind[0]
        return (c_dct, i_search)
   

    # 3
    if ind_bellow != [] and ind == []:
        #print('Ситуация3')
        prev_ind = ind_bellow[0] # Int 
        c_dct[dfc.loc[prev_ind,'control_switch_on']] = [pnt1.ts - pnt2.ts]
        i_search = prev_ind
        return (c_dct, i_search)
    
    # 4.
    if ind_bellow != [] and ind != []:
        #print('Ситуация4')
        # Список индексов
        prev_ind = ind_bellow # list 
        last_ind = [ind[-1]]         
        ind = prev_ind + ind + last_ind
        
        result_df = support_df(pnt1.ts, pnt2.ts, ind, dfc)
        new_ind = result_df.index.values.tolist() 
        # Считаем время управления ручного и автоматического
        for i in new_ind[0:-1]:
            c_dct[result_df.loc[i,'control_switch_on']].append(result_df.loc[i+1,'ts'] - result_df.loc[i,'ts'])
        i_search = last_ind[0]
        return (c_dct, i_search)
    

In [11]:
# cos(d) = sin(φА)·sin(φB) + cos(φА)·cos(φB)·cos(λА − λB),
# где φА и φB — широты, λА, λB — долготы данных пунктов, d — расстояние между пунктами, измеряемое в радианах длиной
# L = d·R, где R = 6371 км — средний радиус земного шара
# для проверки: Москва	55.755773	37.617761, Санкт-Петербург	59.938806	30.314278
# msk = pd.Series([55.755773, 37.617761],index = ['lat', 'lon']) 
# spb = pd.Series([59.938806, 30.314278],index = ['lat', 'lon'])

def distance(pnt1, pnt2):
    R = Decimal(6371210) # Cредний радиус земного шара
    k = Decimal(pi)/Decimal(180) # Перевод в радианы
    cosd = sin(k*pnt1.lat)*sin(k*pnt2.lat) + cos(k*pnt1.lat)*cos(k*pnt2.lat)*cos(k*abs(pnt1.lon - pnt2.lon))
    
    if  cosd>1 or cosd<-1: # защита от погрешностей вроде 1.0000000000000002
        cosd = round(cosd)
                  
    d = acos(cosd)
    distance = Decimal(d)*R   
    return distance

In [12]:
all_dist_dct = {True:0, False:0, None:0}
index_lst = geo_df.index.values.tolist()
index_for_search = -1 # Индекс для навигации по ctrl_df (т.к. он отсортирован)

for i in index_lst[0:-1]:
    pnt1 = geo_df.loc[i,:]
    pnt2 = geo_df.loc[i+1,:]    

    dist_pnt1_pnt2 = distance(pnt1, pnt2)
    time_pnt1_pnt2, index_for_search = time_slots(pnt1, pnt2, ctrl_df, index_for_search)
    t_true = sum(time_pnt1_pnt2[True]) # Можно упростить, если в процедуре считать не в список
    t_false = sum(time_pnt1_pnt2[False]) # Можно упростить, если в процедуре считать не в список
    t_none = sum(time_pnt1_pnt2[None]) # Можно упростить, если в процедуре считать не в список
    
    all_dist_dct[True] += dist_pnt1_pnt2*t_true/(t_true + t_false + t_none)
    all_dist_dct[False] += dist_pnt1_pnt2*t_false/(t_true + t_false + t_none)
    all_dist_dct[None] += dist_pnt1_pnt2*t_none/(t_true + t_false + t_none)
            
print(all_dist_dct)

{True: Decimal('2676.183626275058990815636921'), False: Decimal('273.7120606998044576627037347'), None: Decimal('3.322844952344894409179687500')}
